In [36]:
import pandas as pd

In [37]:
data_movies = pd.read_csv('/Users/andrejzivoj/Documents/datasets/movies.csv')
data_users = pd.read_csv('/Users/andrejzivoj/Documents/datasets/ratings.csv')

display(data_movies.head())
display(data_users.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


# EDA

In [38]:
print(data_movies.isnull().sum())
print(data_users.isnull().sum())

movieId    0
title      0
genres     0
dtype: int64
userId       0
movieId      0
rating       0
timestamp    0
dtype: int64


In [39]:
mean_rating = data_users.groupby(['userId']).agg({'movieId':['count'],'rating':['mean']})
mean_rating.columns = mean_rating.columns.map('_'.join)
mean_rating.reset_index(inplace=True)
display(mean_rating)

mean_watched = data_users.groupby(['movieId']).agg({'userId':['count'],'rating':['mean']})
mean_watched.columns = mean_watched.columns.map('_'.join)
mean_watched.reset_index(inplace=True)
display(mean_watched)

high_rating_film_id = mean_watched[mean_watched.index == mean_watched.rating_mean.idxmax()]['movieId'].values
low_rating_film_id = mean_watched[mean_watched.index == mean_watched.rating_mean.idxmin()]['movieId'].values
print('Movie with high rate')
display(data_movies[data_movies['movieId'] == high_rating_film_id[0]])
print('Movie with low rate')
display(data_movies[data_movies['movieId'] == low_rating_film_id[0]])

,userId,movieId_count,rating_mean
0,1,232,4.366379
1,2,29,3.948276
2,3,39,2.435897
3,4,216,3.555556
4,5,44,3.636364
...,...,...,...
605,606,1115,3.657399
606,607,187,3.786096
607,608,831,3.134176
608,609,37,3.270270


,movieId,userId_count,rating_mean
0,1,215,3.920930
1,2,110,3.431818
2,3,52,3.259615
3,4,7,2.357143
4,5,49,3.071429
...,...,...,...
9719,193581,1,4.000000
9720,193583,1,3.500000
9721,193585,1,3.500000
9722,193587,1,3.500000


Movie with high rate


,movieId,title,genres
48,53,Lamerica (1994),Adventure|Drama


Movie with low rate


,movieId,title,genres
2689,3604,Gypsy (1962),Musical


In [40]:
print(f'All films: {mean_watched.movieId.count()} | All users: {mean_rating.movieId_count.count()}')
print(f'Mean films wathced by user: {mean_rating.movieId_count.mean()}')
print(f'Mean rating from user: {mean_rating.rating_mean.mean()}')
print(f'High rate: {data_movies['title'].loc[data_movies['movieId'] == high_rating_film_id[0]].values[0]} \
| Rate: {mean_watched['rating_mean'].loc[mean_watched.index == mean_watched.rating_mean.idxmax()].values[0]}')
print(f'Low rate:: {data_movies['title'].loc[data_movies['movieId'] == low_rating_film_id[0]].values[0]} \
| Rate: {mean_watched['rating_mean'].loc[mean_watched.index == mean_watched.rating_mean.idxmin()].values[0]}')

All films: 9724 | All users: 610
Mean films wathced by user: 165.30491803278687
Mean rating from user: 3.6572223377474
High rate: Lamerica (1994) | Rate: 5.0
Low rate:: Gypsy (1962) | Rate: 0.5


# Prepare data

In [41]:
import numpy as np
from scipy.sparse import csr_matrix

def create_matrix(df):
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    
    # ID в индексы
    user_mapper = dict(zip(np.unique(df['userId']), list(range(N))))
    movie_mapper = dict(zip(np.unique(df['movieId']), list(range(M))))
    print(user_mapper)
    print(movie_mapper)   

    # Индексы в ID
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df['userId'])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df['movieId']))) 

    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    print(user_index)
    print(movie_index)

    X = csr_matrix((df['rating'], (movie_index, user_index)), shape=(M, N))
                        
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper
                        
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(data_users)

print(X)

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 33: 32, 34: 33, 35: 34, 36: 35, 37: 36, 38: 37, 39: 38, 40: 39, 41: 40, 42: 41, 43: 42, 44: 43, 45: 44, 46: 45, 47: 46, 48: 47, 49: 48, 50: 49, 51: 50, 52: 51, 53: 52, 54: 53, 55: 54, 56: 55, 57: 56, 58: 57, 59: 58, 60: 59, 61: 60, 62: 61, 63: 62, 64: 63, 65: 64, 66: 65, 67: 66, 68: 67, 69: 68, 70: 69, 71: 70, 72: 71, 73: 72, 74: 73, 75: 74, 76: 75, 77: 76, 78: 77, 79: 78, 80: 79, 81: 80, 82: 81, 83: 82, 84: 83, 85: 84, 86: 85, 87: 86, 88: 87, 89: 88, 90: 89, 91: 90, 92: 91, 93: 92, 94: 93, 95: 94, 96: 95, 97: 96, 98: 97, 99: 98, 100: 99, 101: 100, 102: 101, 103: 102, 104: 103, 105: 104, 106: 105, 107: 106, 108: 107, 109: 108, 110: 109, 111: 110, 112: 111, 113: 112, 114: 113, 115: 114, 116: 115, 117: 116, 118: 117, 119: 118, 120: 119, 121: 120, 122: 12

{1: 0, 2: 1, 3: 2, 4: 3, 5: 4, 6: 5, 7: 6, 8: 7, 9: 8, 10: 9, 11: 10, 12: 11, 13: 12, 14: 13, 15: 14, 16: 15, 17: 16, 18: 17, 19: 18, 20: 19, 21: 20, 22: 21, 23: 22, 24: 23, 25: 24, 26: 25, 27: 26, 28: 27, 29: 28, 30: 29, 31: 30, 32: 31, 34: 32, 36: 33, 38: 34, 39: 35, 40: 36, 41: 37, 42: 38, 43: 39, 44: 40, 45: 41, 46: 42, 47: 43, 48: 44, 49: 45, 50: 46, 52: 47, 53: 48, 54: 49, 55: 50, 57: 51, 58: 52, 60: 53, 61: 54, 62: 55, 63: 56, 64: 57, 65: 58, 66: 59, 68: 60, 69: 61, 70: 62, 71: 63, 72: 64, 73: 65, 74: 66, 75: 67, 76: 68, 77: 69, 78: 70, 79: 71, 80: 72, 81: 73, 82: 74, 83: 75, 85: 76, 86: 77, 87: 78, 88: 79, 89: 80, 92: 81, 93: 82, 94: 83, 95: 84, 96: 85, 97: 86, 99: 87, 100: 88, 101: 89, 102: 90, 103: 91, 104: 92, 105: 93, 106: 94, 107: 95, 108: 96, 110: 97, 111: 98, 112: 99, 113: 100, 116: 101, 117: 102, 118: 103, 119: 104, 121: 105, 122: 106, 123: 107, 125: 108, 126: 109, 128: 110, 129: 111, 132: 112, 135: 113, 137: 114, 140: 115, 141: 116, 144: 117, 145: 118, 146: 119, 147: 1

# K-Nearest-Neighbors

In [42]:
from sklearn.neighbors import NearestNeighbors

# Find similar movies using KNN
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbor_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_id]
    k+=1
    knn = NearestNeighbors(n_neighbors=k, algorithm='brute', metric=metric)
    knn.fit(X)
    movie_vec = movie_vec.reshape(1, -1)
    neighbor = knn.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbor.item(i)
        neighbor_ids.append(movie_inv_mapper[n])
    neighbor_ids.pop(0)
    return neighbor_ids

movie_titles = dict(zip(data_movies['movieId'], data_movies['title']))
movie_id=13
similar_ids = find_similar_movies(movie_id, X, k=10)
movie_title = movie_titles[movie_id]
print(f"Если вам понравился фильм: {movie_title}\n")
for i in similar_ids:
    print(movie_titles[i])

Если вам понравился фильм: Balto (1995)

Mighty Aphrodite (1995)
Dead Man Walking (1995)
Leaving Las Vegas (1995)
Primal Fear (1996)
City Hall (1996)
Crucible, The (1996)
Time to Kill, A (1996)
Anne Frank Remembered (1995)
Sense and Sensibility (1995)
Mother (1996)


# Make Recommendation for all movies

In [43]:
rec_movies = []
for movie_id in mean_watched['movieId']:
    try:
        similar_ids = find_similar_movies(movie_id, X, k=10)
        rec_movies.append(similar_ids)
    except:
        rec_movies.append([0])

len(rec_movies)

9724

In [44]:
mean_watched['recommendation_movieId'] = pd.Series(rec_movies)
mean_watched.loc[(mean_watched['rating_mean'] > 3.1) & (mean_watched['rating_mean'] <= 4.4)].head()

,movieId,userId_count,rating_mean,recommendation_movieId
0,1,215,3.920930,"[364, 500, 367, 480, 586, 551, 588, 595, 19, 317]"
1,2,110,3.431818,"[3450, 762, 788, 736, 5, 95, 65, 708, 7, 141]"
2,3,52,3.259615,"[113, 336, 979, 987, 243, 775, 324, 359, 510, 55]"
5,6,102,3.946078,"[708, 5, 62, 141, 852, 3, 648, 17, 11, 539]"
6,7,54,3.185185,"[271, 174, 502, 217, 542, 484, 259, 250, 374, ..."
